******F20DL Coursework Part 1******

**Initialization of Required Libraries**

In [ ]:
import sys
assert sys.version_info >= (3,5)

In [ ]:
import sklearn
assert sklearn.__version__ >= "0.20"

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from pandas.plotting import scatter_matrix

In [ ]:
import time

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB 
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB

In [ ]:
from sklearn.model_selection import cross_val_score

**Importing all coursework Datasets**

In [ ]:
binary_class_x_train=pd.read_csv('./Traffic_Sign/Binary_Classification/x_train_bin.csv')
binary_class_y_train=pd.read_csv('./Traffic_Sign/Binary_Classification/y_train_bin.csv')
binary_class_x_test=pd.read_csv('./Traffic_Sign/Binary_Classification/x_test_bin.csv')
binary_class_y_test=pd.read_csv('./Traffic_Sign/Binary_Classification/y_test_bin.csv')

complete_x_train=pd.read_csv('./Traffic_Sign/CompleteDataSet/x_train_all.csv')
complete_y_train=pd.read_csv('./Traffic_Sign/CompleteDataSet/y_train_all.csv')
complete_x_test=pd.read_csv('./Traffic_Sign/CompleteDataSet/x_test_all.csv')
complete_y_test=pd.read_csv('./Traffic_Sign/CompleteDataSet/y_test_all.csv')

onevsall_y_train_0=pd.read_csv('./Traffic_Sign/OnevrsAll/0_vrs_all/y_train_0.csv')
onevsall_y_train_1=pd.read_csv('./Traffic_Sign/OnevrsAll/1_vrs_all/y_train_1.csv')
onevsall_y_train_2=pd.read_csv('./Traffic_Sign/OnevrsAll/2_vrs_all/y_train_2.csv')
onevsall_y_train_3=pd.read_csv('./Traffic_Sign/OnevrsAll/3_vrs_all/y_train_3.csv')
onevsall_y_train_4=pd.read_csv('./Traffic_Sign/OnevrsAll/4_vrs_all/y_train_4.csv')
onevsall_y_train_5=pd.read_csv('./Traffic_Sign/OnevrsAll/5_vrs_all/y_train_5.csv')
onevsall_y_train_6=pd.read_csv('./Traffic_Sign/OnevrsAll/6_vrs_all/y_train_6.csv')
onevsall_y_train_7=pd.read_csv('./Traffic_Sign/OnevrsAll/7_vrs_all/y_train_7.csv')
onevsall_y_train_8=pd.read_csv('./Traffic_Sign/OnevrsAll/8_vrs_all/y_train_8.csv')
onevsall_y_train_9=pd.read_csv('./Traffic_Sign/OnevrsAll/9_vrs_all/y_train_9.csv')

# Specs assumes 0 is true and 1 is false. So we have to reverse that

def inverse_num(i):
    if i == 1: return 0
    else: return 1

onevsall_y_train_0 = [[inverse_num(i)] for i in onevsall_y_train_0.copy().to_numpy()]
onevsall_y_train_1 = [[inverse_num(i)] for i in onevsall_y_train_1.copy().to_numpy()]
onevsall_y_train_2 = [[inverse_num(i)] for i in onevsall_y_train_2.copy().to_numpy()]
onevsall_y_train_3 = [[inverse_num(i)] for i in onevsall_y_train_3.copy().to_numpy()]
onevsall_y_train_4 = [[inverse_num(i)] for i in onevsall_y_train_4.copy().to_numpy()]
onevsall_y_train_5 = [[inverse_num(i)] for i in onevsall_y_train_5.copy().to_numpy()]
onevsall_y_train_6 = [[inverse_num(i)] for i in onevsall_y_train_6.copy().to_numpy()]
onevsall_y_train_7 = [[inverse_num(i)] for i in onevsall_y_train_7.copy().to_numpy()]
onevsall_y_train_8 = [[inverse_num(i)] for i in onevsall_y_train_8.copy().to_numpy()]
onevsall_y_train_9 = [[inverse_num(i)] for i in onevsall_y_train_9.copy().to_numpy()]

complete_x_train = complete_x_train.astype('float') / 255
complete_x_test = complete_x_test.astype('float') / 255

final_dataset = complete_x_train.copy()
final_dataset['class']=complete_y_train.copy()

**Task 1. Visualization and Initial Data Exploration**

Basic Description of x and y datasets

In [ ]:
complete_x_train.head()

In [ ]:
complete_y_train.head()

In [ ]:
complete_x_train.describe()

In [ ]:
complete_y_train.describe()

In [ ]:
complete_x_train.info()

In [ ]:
complete_y_train.info()

In [ ]:
complete_x_train.shape

In [ ]:
complete_y_train.shape

Visualisation of a single image

In [ ]:
complete_x_train.iloc[0].to_numpy().reshape(48,48)

In [ ]:
def visualize_image(num):
    plt.imshow(complete_x_train.iloc[num].to_numpy().reshape(48,48),cmap="grey")

In [ ]:
visualize_image(0)

Visualisation of an image from each class

In [ ]:
img_toshow = [0,210,2435,6085,7030,8275,8635,8875,9415,9685]

plt.figure(figsize=(10,4))

for i in range (0, len(img_toshow)):
    plt.subplot(2,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(complete_x_train.iloc[img_toshow[i]].to_numpy().reshape(48,48), cmap='gray')
    plt.xlabel("Class "+str(complete_y_train.iloc[img_toshow[i]].to_numpy()[0]))

In [ ]:
plt.figure(figsize=(15,4))
for i in range (0, len(img_toshow)):
    plt.subplot(2,5,i+1)
    plt.grid(False)
    plt.imshow(complete_x_train.iloc[img_toshow[i]].to_numpy().reshape(48,48))
    plt.colorbar()
    plt.grid(False)

Refining and balancing out the dataset to perform better

In [ ]:
temp_dataset = final_dataset.groupby('class', group_keys=False).apply(lambda x: x.sample(frac=0.6))

In [ ]:
(temp_dataset['class'].value_counts()) / len(temp_dataset) * 100

In [ ]:
min_val = temp_dataset['class'].value_counts().min()
balanced_dataset = temp_dataset.groupby('class', group_keys=False).apply(lambda x: x.sample(min_val))
(balanced_dataset['class'].value_counts()) / len(balanced_dataset) * 100

In [ ]:
balanced_dataset_x = balanced_dataset.iloc[:,:2304]
balanced_dataset_y = balanced_dataset.iloc[:,2304:]

Checking correlation with unbalanced and balanced datasets

In [ ]:
# final_corr = final_dataset.corr()
# final_corr['class'].sort_values(ascending=False)[0:10]

In [ ]:
# balanced_corr = balanced_dataset.corr()
# balanced_corr['class'].sort_values(ascending=False)[0:10]

**Naïve Bayes Classifier**

In [ ]:
def run_naivebayes_print(classifier, dataset_train_x, dataset_train_y, dataset_test_x, dataset_test_y):
    clf = None
    if classifier == "gaussian":
        clf = GaussianNB()
    elif classifier == "multinomial":
        clf = MultinomialNB()
    elif classifier == "complement":
        clf = ComplementNB()
    elif classifier == "categorical":
        clf = CategoricalNB(min_categories=10)
    else:
        clf = None
    clf.fit(dataset_train_x,dataset_train_y)
    complete_y_pred = clf.predict(dataset_test_x)
    print("Accuracy:", accuracy_score(dataset_test_y, complete_y_pred))
    print("Precision: " , precision_score(dataset_test_y, complete_y_pred, average='weighted'))
    print('Recall score: ', recall_score(dataset_test_y, complete_y_pred, average='weighted'))
    print('F-measure: ', f1_score(dataset_test_y, complete_y_pred, average='weighted'))
    print(classification_report(dataset_test_y, complete_y_pred))
    conf_matrix = confusion_matrix(dataset_test_y, complete_y_pred)
    class_names = list(set(dataset_test_y) | set(complete_y_pred))
    cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
    cm_display.plot()
    plt.show()
    tpr = conf_matrix[1,1]/(conf_matrix[1,1]+conf_matrix[1,0])
    fpr = conf_matrix[0,0]/(conf_matrix[0,0]+conf_matrix[0,1])
    print("True Positive Rate: ", tpr)
    print("False Positive Rate: ", fpr)

def run_naivebayes(classifier, dataset_train_x, dataset_train_y, dataset_test_x, dataset_test_y):
    clf = None
    ret_arr=[]
    if classifier == "gaussian":
        clf = GaussianNB()
    elif classifier == "multinomial":
        clf = MultinomialNB()
    elif classifier == "complement":
        clf = ComplementNB()
    elif classifier == "categorical":
        clf = CategoricalNB(min_categories=10)
    else:
        clf = None
    clf.fit(dataset_train_x,dataset_train_y)
    complete_y_pred = clf.predict(dataset_test_x)
    ret_arr = [accuracy_score(dataset_test_y, complete_y_pred), 
            precision_score(dataset_test_y, complete_y_pred, average='weighted'), 
            recall_score(dataset_test_y, complete_y_pred, average='weighted'),
            f1_score(dataset_test_y, complete_y_pred, average='weighted'),
            classification_report(dataset_test_y, complete_y_pred),
            confusion_matrix(dataset_test_y, complete_y_pred)
            ]
    return ret_arr
    

Gaussian Naive Bayes Classifier using complete dataset

In [ ]:
run_naivebayes_print("gaussian",complete_x_train,complete_y_train,complete_x_test,complete_y_test)

Multinomial Naive Bayes Classifier using complete dataset

In [ ]:
run_naivebayes_print("multinomial",complete_x_train,complete_y_train,complete_x_test,complete_y_test)

Complement Naive Bayes Classifier using complete dataset

In [ ]:
run_naivebayes_print("complement",complete_x_train,complete_y_train,complete_x_test,complete_y_test)

Categorical Naive Bayes Classifier using complete dataset

In [ ]:
run_naivebayes_print("categorical",complete_x_train,complete_y_train,complete_x_test,complete_y_test)

Gaussian Naive Bayes Classifier using stratified dataset

In [ ]:
run_naivebayes_print("gaussian",balanced_dataset_x,balanced_dataset_y,complete_x_test,complete_y_test)

Multinomial Naive Bayes Classifier using stratified dataset

In [ ]:
run_naivebayes_print("multinomial",balanced_dataset_x,balanced_dataset_y,complete_x_test,complete_y_test)

Complement Naive Bayes Classifier using stratified dataset

In [ ]:
run_naivebayes_print("complement",balanced_dataset_x,balanced_dataset_y,complete_x_test,complete_y_test)

Categorical Naive Bayes Classifier using stratified dataset

In [ ]:
run_naivebayes_print("categorical",balanced_dataset_x,balanced_dataset_y,complete_x_test,complete_y_test)

Getting Dataset using PCA

In [ ]:
from sklearn.decomposition import PCA 
pca = PCA(n_components = len(balanced_dataset_x))
X2D = pca.fit_transform(balanced_dataset_x)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

In [ ]:
pca = PCA(n_components = d)
X_reduced = pd.DataFrame(pca.fit_transform(balanced_dataset_x))

In [ ]:
pca_dataset_x = pd.DataFrame(pca.inverse_transform(X_reduced))
pca_dataset_x[pca_dataset_x < 0] = 0
pca_dataset_y = balanced_dataset_y.copy()

In [ ]:
gaussianclf = GaussianNB()
multinomialclf = MultinomialNB()
complementclf = ComplementNB()
categoricalclf = CategoricalNB(min_categories=10) 

Gaussian Naive Bayes Classifier using PCA recovered dataset

In [ ]:
run_naivebayes_print("gaussian",pca_dataset_x,pca_dataset_y,complete_x_test,complete_y_test)

Multinomial Naive Bayes Classifier using PCA recovered dataset

In [ ]:
run_naivebayes_print("multinomial",pca_dataset_x,pca_dataset_y,complete_x_test,complete_y_test)

Complement Naive Bayes Classifier using PCA recovered dataset

In [ ]:
run_naivebayes_print("complement",pca_dataset_x,pca_dataset_y,complete_x_test,complete_y_test)

Categorical Naive Bayes Classifier using PCA recovered dataset

In [ ]:
run_naivebayes_print("categorical",pca_dataset_x,pca_dataset_y,complete_x_test,complete_y_test)

**Selecting best correlating features**

Methods to select the top correlating features

In [ ]:
class_arr_data = [
                onevsall_y_train_0.copy(),
                onevsall_y_train_1.copy(),
                onevsall_y_train_2.copy(),
                onevsall_y_train_3.copy(),
                onevsall_y_train_4.copy(),
                onevsall_y_train_5.copy(),
                onevsall_y_train_6.copy(),
                onevsall_y_train_7.copy(),
                onevsall_y_train_8.copy(),
                onevsall_y_train_9.copy()
            ]

x_data = complete_x_train.copy()

In [ ]:
def getTopFeatureMethod1(x):
    return_arr = []
    for i in class_arr_data:
        bestfeatures = SelectKBest(score_func=chi2, k=x//len(class_arr_data))
        fit = bestfeatures.fit(x_data,i)
        dfscores = pd.DataFrame(fit.scores_)
        dfcolumns = pd.DataFrame(x_data.columns)
        featureScores = pd.concat([dfcolumns,dfscores],axis=1)
        featureScores.columns = ['Specs','Score']
        return_arr = np.unique(np.concatenate((return_arr,featureScores.nlargest(x//len(class_arr_data),'Score')['Specs']),0))
    return return_arr

In [ ]:
def getTopFeatureMethod2(x):
    features_array = [[]]*10
    return_arr = [[]]*3
    for i in range (0,len(class_arr_data)):
        bestfeatures = SelectKBest(score_func=chi2, k=x//len(class_arr_data))
        fit = bestfeatures.fit(x_data,class_arr_data[i])
        dfscores = pd.DataFrame(fit.scores_)
        dfcolumns = pd.DataFrame(x_data.columns)
        featureScores = pd.concat([dfcolumns,dfscores],axis=1)
        featureScores.columns = ['Specs','Score']
        features_array[i]=featureScores.nlargest(x//len(class_arr_data),'Score')['Specs']
    for i in range (0,10):
        return_arr[0] = np.unique(np.concatenate((return_arr[0], features_array[i][0:x//len(class_arr_data)]),0))
        return_arr[1] = np.unique(np.concatenate((return_arr[1], features_array[i][0:x//(len(class_arr_data)*2)]),0))
        return_arr[2] = np.unique(np.concatenate((return_arr[2], features_array[i][0:x//(len(class_arr_data)*4)]),0))
    return return_arr

In [ ]:
start_time = time.time()
temp1 = getTopFeatureMethod1(200)
temp2 = getTopFeatureMethod1(100)
temp3 = getTopFeatureMethod1(75)
print("Time taken to compute best features using method 1 : ",time.time()-start_time)

start_time = time.time()
temp = getTopFeatureMethod2(200)
print("Time taken to compute best features using method 2 : ",time.time()-start_time)

In [ ]:
dataset1_method_1 = balanced_dataset_x.copy()[temp1]
dataset2_method_1 = balanced_dataset_x.copy()[temp2]
dataset3_method_1 = balanced_dataset_x.copy()[temp3]

dataset1_method_2 = balanced_dataset_x.copy()[temp[0]]
dataset2_method_2 = balanced_dataset_x.copy()[temp[1]]
dataset3_method_2 = balanced_dataset_x.copy()[temp[2]]

dataset1_test_method_1 = complete_x_test.copy()[temp1]
dataset2_test_method_1 = complete_x_test.copy()[temp2]
dataset3_test_method_1 = complete_x_test.copy()[temp3]

dataset1_test_method_2 = complete_x_test.copy()[temp[0]]
dataset2_test_method_2 = complete_x_test.copy()[temp[1]]
dataset3_test_method_2 = complete_x_test.copy()[temp[2]]

**Running Naïve Bayes Classifier on the datasets produced by running the methods mentioned in above section**

In [ ]:
datasets = [
    [(dataset1_method_1, dataset1_test_method_1), (dataset2_method_1, dataset2_test_method_1), (dataset3_method_1, dataset3_test_method_1)],
    [(dataset1_method_2, dataset1_test_method_2), (dataset2_method_2, dataset2_test_method_2), (dataset3_method_2, dataset3_test_method_2)]
]

bayes = ["gaussian", "multinomial", "complement", "categorical"]

report = [[[[]]*len(bayes)]*len(datasets[0])]*len(datasets)

for i in range (0,len(datasets)):
    for j in range (0,len(datasets[i])):
        for k in range (0,len(bayes)):
            report[i][j][k] = run_naivebayes(bayes[k], datasets[i][j][0], balanced_dataset_y, datasets[i][j][1], complete_y_test)
            report[i][j][k].append(bayes[k].capitalize())

def gen_report(report):
    for i in range (0,len(report)):
        print("*"*100)
        print("Using Method ",i+1)
        print("*"*100)
        acc=[]
        prec=[]
        recall=[]
        f1=[]
        dataset=[]
        baye=[]
        for j in range (0,len(report[i])):
            for k in range (0,len(report[i][j])):
                if(report[i][j]==[]):
                    continue
                acc.append(report[i][j][k][0])
                prec.append(report[i][j][k][1])
                recall.append(report[i][j][k][2])
                f1.append(report[i][j][k][3])
                dataset.append(j+1)
                baye.append(report[i][j][k][6])
        temp_table = {"Dataset":dataset,"Naive Bayes":baye,"Accuracy":acc,"Precision":prec,"Recall Score":recall,"F1 Score":f1}
        print(pd.DataFrame(temp_table))
        print()

gen_report(report)


In [ ]:
pip install cv2

In [ ]:
import cv2
x_train_sm = np.apply_along_axis(
    func1d=lambda img: cv2.resize(img.reshape(48, 48), dsize=(16, 16)),
    axis=1, arr=complete_x_train).reshape(-1, 16*16)